### **Import Library**
Kita mulai dengan mengimpor semua library yang diperlukan untuk memproses dan menggabungkan data:
- `os` & `glob`: untuk membaca file dari folder
- `pandas` & `numpy`: untuk manipulasi data
- `statistics.mode`: untuk menghitung modus (nilai paling sering muncul)

In [12]:
# Import library yang dibutuhkan
import os
import pandas as pd
import numpy as np
import glob
from statistics import mode

### **Membaca dan Menggabungkan Data CO2**

Kode ini secara sistematis melakukan beberapa tugas penting:
1.  **Menemukan File:** Menggunakan `glob` untuk secara otomatis menemukan semua file dengan ekstensi `.csv` di dalam folder data mentah CO2.
2.  **Membaca & Menggabungkan:** Membaca setiap file CSV yang ditemukan ke dalam DataFrame, lalu langsung menggabungkannya menjadi satu DataFrame tunggal (`df_co2`) menggunakan `pd.concat`.
3.  **Memilih Kolom:** Hanya menyimpan kolom-kolom yang relevan, yaitu `timestamp` dan `co2`.
4.  **Konversi Tipe Data:** Memastikan kolom `timestamp` diubah menjadi format `datetime` dan kolom `co2` menjadi format numerik (`float` atau `int`).

**Tujuan:**
Untuk mengonsolidasikan semua data CO2 yang mungkin terfragmentasi dalam banyak file (misalnya, satu file per hari) menjadi satu dataset tunggal yang siap untuk diolah lebih lanjut.

In [13]:
# Tentukan path ke folder data CO2
folder_path_co2 = 'dataset/data-raw/co2'
all_csv_files = glob.glob(os.path.join(folder_path_co2, "*.csv"))

# Gabungkan semua file CSV
df_co2 = pd.concat([pd.read_csv(f) for f in all_csv_files], ignore_index=True)
df_co2 = df_co2[['timestamp', 'co2']]

# Konversi tipe data
df_co2['timestamp'] = pd.to_datetime(df_co2['timestamp'], errors='coerce')
df_co2['co2'] = pd.to_numeric(df_co2['co2'], errors='coerce')

**Hasil dan Insight:** 
- Terbentuknya sebuah DataFrame tunggal bernama `df_co2` yang berisi seluruh data CO2 dari semua file sumber.
-   Penggunaan `glob` membuat proses ini **otomatis dan skalabel**. Jika ada file data baru yang ditambahkan ke folder, kode ini akan langsung menyertakannya pada eksekusi berikutnya tanpa perlu diubah.
-   Penggunaan parameter **`errors='coerce'`** saat konversi tipe data adalah langkah "jaga-jaga". Jika ada baris data yang formatnya salah atau rusak di file asli, proses tidak akan berhenti karena error. Sebaliknya, data yang salah itu akan diubah menjadi `NaN` (Not a Number) atau `NaT` (Not a Time), yang bisa kita tangani secara sistematis di tahap pembersihan selanjutnya.

### **Sampling dan Data CO2**

Prosesnya terdiri dari empat bagian:
1.  **Standardisasi Waktu:** Membuat kolom baru `minute` dengan membulatkan `timestamp` ke bawah ke awal menit terdekat (`.floor('min')`). Ini mengelompokkan semua pembacaan dalam satu menit ke stempel waktu yang sama.
2.  **Agregasi per Menit (Resampling):** Mengelompokkan data berdasarkan kolom `minute` yang baru, lalu untuk setiap menit, mengambil nilai **modus (mode)** atau nilai yang paling sering muncul dari pembacaan CO2.
3.  **Membuat Indeks Waktu Penuh:** Membuat sebuah rentang waktu yang ideal dan lengkap (`full_minutes`) dari awal hingga akhir data, dengan interval persis setiap satu menit tanpa ada yang terlewat.
4.  **Penggabungan (Merge):** Melakukan `left merge` antara indeks waktu yang lengkap dengan data CO2 yang sudah di-resample.

**Tujuan:**
Untuk mengatasi dua masalah umum pada data sensor:
-   Adanya beberapa pembacaan dalam satu menit.
-   Adanya jeda waktu di mana tidak ada pembacaan sama sekali.
Dengan proses ini, kita memastikan kita memiliki satu nilai representatif untuk setiap menit dan struktur data yang konsisten (satu baris per menit), yang sangat penting untuk penggabungan dengan data lain dan untuk analisis deret waktu.

In [14]:
# Bulatkan waktu ke menit terdekat
df_co2['minute'] = df_co2['timestamp'].dt.floor('min')

# Sampling modus CO2 per menit
minute_co2 = df_co2.groupby('minute')['co2'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
).reset_index()

# Lengkapi semua menit agar tidak ada yang hilang
full_minutes = pd.date_range(
    start=df_co2['minute'].min().floor('D'),
    end=df_co2['minute'].max().ceil('D') - pd.Timedelta(minutes=1),
    freq='1min'
)
full_minutes_df = pd.DataFrame({'minute': full_minutes})
co2_sampled = full_minutes_df.merge(minute_co2, on='minute', how='left')

**Hasil dan Insight:**
-   Terbentuknya DataFrame `co2_sampled` di mana setiap baris mewakili satu menit yang unik dan berurutan. Jika pada satu menit tertentu tidak ada data CO2 asli, nilainya akan menjadi `NaN` (kosong).
-   Keputusan untuk menggunakan **modus (`.mode()`)** daripada rata-rata (`.mean()`) adalah pilihan yang dirasa tepat. Jika dalam satu menit ada beberapa pembacaan dan salah satunya adalah *noise* atau nilai yang sangat aneh, modus tidak akan terpengaruh, sementara rata-rata akan menjadi tidak akurat. Ini membuat prosesnya lebih kuat (robust) terhadap data pencilan.

### **Menyimpan Data CO2 Hasil Sampling**

DataFrame `co2_sampled` yang berisi data CO2 yang sudah rapi dan berurutan per menit, disimpan ke dalam sebuah file CSV baru.

**Tujuan:**
Langkah ini berfungsi sebagai **checkpoint** atau titik simpan yang sangat penting. Proses resampling pada langkah sebelumnya bisa jadi memakan waktu jika datanya sangat besar. Dengan menyimpan hasilnya ke dalam file, kita mengamankan progres kita. Jika terjadi error di langkah-langkah selanjutnya, kita bisa memulai kembali dari titik ini dengan hanya memuat file `co2_per_minute.csv`, tanpa harus mengulang seluruh proses resampling dari awal.

In [15]:
# Simpan hasil sampling CO2
output_co2_path = 'dataset/data-clean/co2_per_minute.csv'
co2_sampled.to_csv(output_co2_path, index=False)
print(f"CO2 per menit disimpan di: {output_co2_path}")

CO2 per menit disimpan di: dataset/data-clean/co2_per_minute.csv


**Hasil dan Insight:**
-   Berhasil dibuatnya file `dataset/data-clean/co2_per_minute.csv`.
-   Kita sekarang memiliki file data CO2 yang bersih dan terstruktur secara independen dari file-file mentah aslinya. Ini membuat alur kerja kita lebih modular dan efisien.

### **Memuat dan Membersihkan Data Cuaca**

Serupa dengan penanganan data CO2, kode ini melakukan beberapa tugas untuk data cuaca:
1.  **Mencari dan Membaca File:** Menggunakan `os.listdir` dan sebuah *loop* untuk membaca semua file CSV dari folder data mentah cuaca.
2.  **Menggabungkan:** Menyusun semua data cuaca dari file-file terpisah menjadi satu DataFrame tunggal (`df_weather`) menggunakan `pd.concat`.
3.  **Membuang Kolom:** Menghapus kolom-kolom yang tidak akan digunakan dalam analisis ini (`direction`, `angle`, `wind_speed`) dengan metode `.drop()`.
4.  **Konversi Tipe Data:** Memastikan kolom `timestamp` diubah menjadi format `datetime` yang benar.

**Tujuan:**
Untuk mengonsolidasikan dan membersihkan semua data cuaca, sehingga siap untuk digabungkan dengan data CO2 yang telah kita proses sebelumnya.

In [16]:
folder_path_weather = 'dataset/data-raw/cuaca'
file_list_weather = sorted(os.listdir(folder_path_weather))

df_list_weather = []
for file in file_list_weather:
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path_weather, file))
        df_list_weather.append(df)

df_weather = pd.concat(df_list_weather, ignore_index=True)
df_weather = df_weather.drop(['direction', 'angle', 'wind_speed'], axis=1)  # Hapus kolom tak diperlukan
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'])

**Hasil dan Insight:**
-   Terbentuknya sebuah DataFrame tunggal bernama `df_weather` yang berisi semua data cuaca yang relevan (`temperature`, `humidity`, `rainfall`, `pyrano`) dengan stempel waktu yang sudah diformat dengan benar.
-   Tindakan menghapus beberapa kolom cuaca merupakan bentuk awal dari **seleksi fitur (feature selection)**. Ini menunjukkan keputusan sadar bahwa fitur-fitur tersebut dianggap tidak terlalu relevan untuk tujuan akhir proyek, sehingga model dan analisis kita bisa lebih fokus pada variabel yang paling penting.

### **Memfilter dan Merapikan Data Cuaca**

Kode ini melakukan dua tugas penting pada data cuaca:
1.  **Memfilter Berdasarkan Tanggal:** Pertama, data cuaca disaring untuk hanya menyertakan data dalam rentang tanggal tertentu (dari 24 April hingga 7 Mei 2025).
2.  **Regularisasi Waktu:** Sama seperti pada data CO2, kita membuat sebuah rentang waktu (`full_range`) yang lengkap untuk setiap menit dalam periode tersebut, lalu menggabungkannya (`left merge`) dengan data cuaca yang sudah difilter.

**Tujuan:**

-   **Scoping:** Memastikan kita hanya bekerja dengan data dari periode yang relevan, kemungkinan untuk menyamakannya dengan ketersediaan data CO2.
-   **Alignment:** Memaksa data cuaca untuk memiliki struktur waktu yang **identik** dengan data CO2 yang telah kita proses sebelumnya (satu baris per menit, tanpa ada jeda). Ini adalah persiapan krusial untuk langkah penggabungan akhir.

In [17]:
start_date = '2025-04-24'
end_date = '2025-05-07'

# Filter tanggal
filtered_weather = df_weather[(df_weather['timestamp'] >= start_date) & (df_weather['timestamp'] <= end_date)].copy()

# Lengkapi waktu setiap menit
full_range = pd.date_range(start=start_date + ' 00:00:00', end=end_date + ' 23:59:00', freq='min')
full_weather_df = pd.DataFrame({'timestamp': full_range})

weather_sampled = pd.merge(full_weather_df, filtered_weather, on='timestamp', how='left')

**Hasil dan Insight:**
-   Terbentuknya DataFrame `weather_sampled` yang berisi data cuaca hanya untuk rentang tanggal yang ditentukan dan memiliki struktur waktu per menit yang sempurna. Baris di mana tidak ada data cuaca asli akan berisi nilai `NaN`.
-   Dengan menyamakan "grid" waktu untuk kedua dataset (CO2 dan cuaca) sebelum menggabungkannya, kita memastikan integritas data. Proses penggabungan akhir akan jauh lebih bersih dan akurat karena kedua dataset sudah "berbicara dalam bahasa waktu yang sama".

### **Menyimpan Data Cuaca Hasil Olahan (Checkpoint)**

DataFrame `weather_sampled` yang berisi data cuaca yang sudah difilter dan dirapikan per menit, disimpan ke dalam sebuah file CSV baru.

**Tujuan:**
Sama seperti pada data CO2, langkah ini berfungsi sebagai **checkpoint**. Kita menyimpan hasil pengolahan data cuaca yang sudah bersih ke dalam file terpisah. Ini membuat alur kerja kita menjadi sangat **modular**. Jika kita perlu melakukan penggabungan ulang nanti, kita bisa langsung memuat file ini dan file CO2 yang sudah diproses, tanpa perlu mengulang semua langkah pembersihan dari awal.

In [18]:
output_weather_path = 'dataset/data-clean/cuaca_per_menit.csv'
weather_sampled.to_csv(output_weather_path, index=False)
print(f"Cuaca per menit disimpan di: {output_weather_path}")

Cuaca per menit disimpan di: dataset/data-clean/cuaca_per_menit.csv


**Hasil dan Insight:**
-   Berhasil dibuatnya file `dataset/data-clean/cuaca_per_menit.csv`.
-   Pada titik ini, kita telah berhasil memproses dua sumber data yang berbeda (CO2 dan cuaca) secara terpisah dan menyimpannya dalam format yang bersih dan terstruktur. Keduanya kini siap untuk "dipertemukan" pada langkah penggabungan final.

### **Menggabungkan Data CO2 dan Cuaca yang Telah Diproses**

1.  **Memuat Data Checkpoint:** Kita memuat kembali dua file CSV bersih yang telah kita buat sebelumnya: `co2_per_minute.csv` dan `cuaca_per_menit.csv`.
2.  **Konversi Waktu:** Memastikan kolom waktu di kedua DataFrame (`minute` dan `timestamp`) berformat `datetime`.
3.  **Penggabungan (`Merge`):** Menggabungkan kedua DataFrame menjadi satu (`merged_df`) menggunakan `pd.merge`. Kunci penggabungannya adalah kolom waktu yang sudah kita samakan strukturnya pada langkah-langkah sebelumnya.
4.  **Finalisasi Kolom:** Membersihkan kolom-kolom sisa dari proses merge, yaitu menghapus satu kolom waktu yang duplikat dan mengganti nama kolom waktu utama menjadi `timestamp`.

**Tujuan:**
Untuk menciptakan satu **master dataset** yang tunggal. Setiap baris dalam dataset final ini akan berisi informasi lengkap dan pembacaan CO2 beserta kondisi cuaca yang terjadi pada menit yang sama.

In [19]:
# Load ulang data yang telah disampling
co2_df = pd.read_csv('dataset/data-clean/co2_per_minute.csv')
cuaca_df = pd.read_csv('dataset/data-clean/cuaca_per_menit.csv')

# Konversi waktu agar bisa digabung
co2_df['minute'] = pd.to_datetime(co2_df['minute'])
cuaca_df['timestamp'] = pd.to_datetime(cuaca_df['timestamp'])

# Gabungkan berdasarkan waktu
merged_df = pd.merge(co2_df, cuaca_df, left_on='minute', right_on='timestamp', how='left')

# Finalisasi
merged_df.drop(columns=['timestamp'], inplace=True)
merged_df.rename(columns={'minute': 'timestamp'}, inplace=True)

**Hasil dan Insight:**
-   Terbentuknya DataFrame `merged_df` yang merupakan produk akhir dari notebook ini, siap untuk disimpan dan digunakan untuk analisis selanjutnya.
-   Keberhasilan dan kesederhanaan dari `pd.merge` di tahap ini adalah buah dari kerja keras pada langkah-langkah sebelumnya. Karena kita sudah melakukan **regularisasi dan alignment data** pada kedua dataset, proses penggabungan akhir menjadi sangat lugas dan akurat. Ini menunjukkan pentingnya menyiapkan setiap sumber data secara terpisah sebelum menggabungkannya.

### **Menyimpan Data Gabungan Final**

Ini adalah langkah final dari notebook ini. DataFrame `merged_df` yang berisi gabungan lengkap data CO2 dan cuaca disimpan ke dalam sebuah file CSV baru, yaitu `data_output_collecting.csv`. Parameter `index=False` digunakan agar indeks DataFrame tidak ikut ditulis ke dalam file.

**Tujuan:**
Untuk menghasilkan **artefak** atau *output* akhir dari tahap pengumpulan dan penggabungan data. File CSV ini akan menjadi **sumber data tunggal (single source of truth)** untuk semua notebook dan analisis di tahap selanjutnya (EDA dan *modeling*).

In [20]:
output_merged_path = 'dataset/data-clean/data_output_collecting.csv'
merged_df.to_csv(output_merged_path, index=False)
print(f"Data gabungan berhasil disimpan di: {output_merged_path}")

Data gabungan berhasil disimpan di: dataset/data-clean/data_output_collecting.csv


**Hasil dan Insight:**
-   Berhasil dibuatnya file `dataset/data-clean/data_output_collecting.csv`.
-   Dengan disimpannya file ini, maka seluruh proses penggabungan data telah selesai.